In [1]:
!conda install -c conda-forge geopy --yes        # if needed
!conda install -c conda-forge folium=0.5.0 --yes # if needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import xml
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # transform json file into pandas dataframe

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
data_orig = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")#, attrs = {'id': 'table'})

# get the table
data_orig = data_orig[0]
data_orig.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [3]:
data = data_orig.copy(deep=True)
data.shape

(287, 3)

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
data.columns = ['PostalCode', 'Borough', 'Neighborhood']
data = data[data['Borough']!='Not assigned']

# If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough
neighborhoods_to_replace = data['Neighborhood']=='Not assigned'
data.loc[neighborhoods_to_replace, 'Neighborhood'] = data.loc[neighborhoods_to_replace, 'Borough']

In [5]:

def aggregate_neighbourhoods(df):
    # create a dictionary to hold the postal codes as keys and (Boroughs, Neighborhoods) as values
    D = dict.fromkeys(data['PostalCode'].unique())
    # loop through the rows of the dataframe df
    for item in df:
        # get the list of Neighborhoods and join the list items with ", "
        hds = ", ".join(list(item[1]['Neighborhood'].unique()))
        # now get the list of boroughs and join the list items with ", "
        bor = ", ".join(list(item[1]['Borough'].unique()))
        # populate the dictionary
        D[item[0]] = (bor, hds)
        
    new_df = pd.DataFrame.from_dict(D, orient='index')  # without orient='index' the dict keys will be the columns of new_df
    new_df.reset_index(inplace=True)  # reset the index to row numbers instead of postal codes
    new_df.columns = ['Postal Code', 'Borough', 'Neighborhood']  # name the columns
    return new_df


df = data.groupby(['PostalCode'])  # group the data by the postal codes
data = aggregate_neighbourhoods(df)  # pass the dataframe in the function to get the final dataframe
data.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
data.shape

(103, 3)

In [9]:
geo_coords = pd.read_csv('http://cocl.us/Geospatial_data')

# sanity check. The below should not return an error if the dataframe data is correct
assert(geo_coords.shape[0] == data.shape[0])

geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
analysis_data = data.merge(geo_coords, on='Postal Code')
analysis_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [11]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [12]:
kclusters = 7

analysis_data_clustering = analysis_data.drop(['Postal Code', 'Borough', 'Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(analysis_data_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 6, 1, 6, 0, 4, 3, 2, 6], dtype=int32)